In [1]:
from hdmf.common import ExternalResources
from hdmf import Container, Data
import pandas as pd

In [2]:
er=ExternalResources(name='test') #This will be our instance of the ExternalResources class

### add_ref

add_ref will be one of the class methods that simplifies how to add new data to multiple tables. You can think of add_ref as a wrapper function.

In [3]:
data = Data(name= "species", data='homo sapien')

In [4]:
er.add_ref(container = data, field='', key='one',resource_name='new_name',resource_uri='new_uri',entity_id='111',entity_uri='entity_uri1')

This method allows you to either reference or add new entries into all the Tables for ExternalResources. The function takes in the keyword arguments and will automatically create new entries into Tables that don't have the input parameters. In our example above, all of our Tables were empty before add_ref, so calling the method will create new rows of data in every Table. Note that if you want to reference objects then you need to reference the id via a string, but if you want to reference a key then you reference the key object.

In [3]:
er.add_key(key_name='two')

In [6]:
key_object = er.get_key(key_name='two') #This is the method that returns a key object.
object_id = er.objects.data[0][0] 

In [7]:
er.add_ref(container = object_id, field='', key=key_object, resource_name='new_name_2',resource_uri='new_uri2',entity_id='112',entity_uri='entity_uri2')

We can see these additions by using the "to_dataframe" method that ExternalResources inherits.

In [8]:
er.resources.to_dataframe()

,name,resource_uri
0,new_name,new_uri
1,new_name_2,new_uri2


In [9]:
er.entities.to_dataframe()

,keytable_idx,resource_table_idx,entity_id,entity_uri
0,0,0,111,entity_uri1
1,1,1,112,entity_uri2


In [10]:
er.keys.to_dataframe()

,key_name
0,one
1,two


In [11]:
er.objects.to_dataframe()

,object_id,field
0,99c08410-7569-412a-9efb-931087703628,


In [12]:
er.object_keys.to_dataframe()

,objecttable_idx,keytable_idx
0,0,0
1,0,1


### add_key

This method creates a new key object and adds it to the KeyTable.

In [12]:
k=er.add_key(key='three')
er.keys.to_dataframe()

,key_name
0,one
1,two
2,three


### Add_Resource

This method creates new resource objects and adds them to the ResourceTable. The ResourceTable holds the name and uri for the resource. For example, the user you could add NCBI Taxonomy with its uri.

In [13]:
r=er.add_resource(name='taxonomy',uri='uri')

In [14]:
er.resources.to_dataframe()

,name,resource_uri
0,new_name,new_uri
1,new_name_2,new_uri2
2,taxonomy,uri


### Add_entity

This method creates a new entity object and adds it to the EntityTable. Keeping with the NCBI Taxonomy example, the entity would be a specfic item or "search" in NCBI Taxonomy, whereas the resource is the NCBI Taxonomy itself.

In [15]:
er.add_entity(key='four',resource_table_idx=r,entity_id='113',entity_uri='entity_uri3')

In [16]:
er.entities.to_dataframe()

,keytable_idx,resource_table_idx,entity_id,entity_uri
0,0,0,111,entity_uri1
1,1,1,112,entity_uri2
2,3,2,113,entity_uri3


### Add_Object

This method creates a new object and adds it to the ObjectTable.

In [17]:
data = Data(name= "species", data='Elaphe obsoleta obsoleta ')

In [18]:
o=er.add_object(container=data,field='')

In [19]:
er.objects.to_dataframe()

,object_id,field
0,13e5cdcf-1a87-448d-a09b-dbf2061fced9,
1,0492abca-7ca9-49bb-8c14-cf658c139477,


### Add_ExternalReference (adds objectkey to objectkeytable)

In [20]:
object_ = er._check_object_field(data,field="") #This method returns an Object object.
key_object = er.get_key(key_name='one')

In [21]:
er.add_external_reference(object_,key_object)

In [22]:
er.object_keys.to_dataframe()

,objecttable_idx,keytable_idx
0,0,0
1,0,1
2,2,0


In [23]:
er.entities.to_dataframe()

,keytable_idx,resource_table_idx,entity_id,entity_uri
0,0,0,111,entity_uri1
1,1,1,112,entity_uri2
2,3,2,113,entity_uri3


### add_keys

This is another wrapper function, but allows the use of a pandas DataFrame to add/reference keys, resources, and entities.

In [4]:
new_data={'key_name':'two',
             'resources_idx':1,
             'entity_id':'12345',
             'entity_uri':'entity_uri4'}

In [5]:
df=pd.DataFrame(new_data,index=[0])
df

,key_name,resources_idx,entity_id,entity_uri
0,two,1,12345,entity_uri4


In [6]:
er.add_keys(df)

{'two': <hdmf.common.resources.Key at 0x7f865d069750>}

In [7]:
er.resources.to_dataframe()

,resource,resource_uri


In [19]:
er.entities.to_dataframe()

,keytable_idx,resource_table_idx,entity_id,entity_uri
0,0,0,111,entity_uri1
1,1,1,112,entity_uri2


In [17]:
er.keys.to_dataframe()

,key_name
0,one
1,two


In [34]:
ret = dict()
ret['key']=er.add_key('three')
ret


{'key': <hdmf.common.resources.Key at 0x7f8051107f90>}

### Get_Keys

This method returns a DataFrame with information about keys used to make references to external resources.

In [5]:
er.get_keys([er.get_key('key1'), er.get_key('key2')])

,key_name,resources_idx,entity_id,entity_uri
0,key1,0,id11,url11
1,key1,1,id12,url12
2,key2,2,id13,url13


In [3]:
resource1 = er.add_resource(resource='resource1', uri='resource_uri1')
resource2 = er.add_resource(resource='resource2', uri='resource_uri2')
resource3 = er.add_resource(resource='resource3', uri='resource_uri3')

keys = pd.DataFrame(
            data=[['key1', resource1, "id11", 'url11'],
                  ['key1', resource2, "id12", 'url12'],
                  ['key2', resource3, "id13", 'url13']],
            columns=['key_name', 'resources_idx', 'entity_id', 'entity_uri'])

In [4]:
er.add_keys(keys)

{'key1': <hdmf.common.resources.Key at 0x7f9e2a174210>,
 'key2': <hdmf.common.resources.Key at 0x7f9e29e3f810>}

In [6]:
er = ExternalResources(name='new_example')

# The add_key method will create a new key
er.add_key(key_name='human')
er.add_key(key_name='gorilla')

# The get_key method will return the key object of the key_name.
key_object = er.get_key(key_name='gorilla')

# This will return a pandas dataframe of the KeyTable
er.keys.to_dataframe()

###############################################################################
# Using the add_keys method
# ------------
# This is another wrapper function, but allows the use of a pandas DataFrame
# to add/reference keys, resources, and entities.

new_data = {
    'key_name': 'human',
    'resources_idx': 0,
    'entity_id': 'NCBI:txid9606',
    'entity_uri': 'https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606'}

df = pd.DataFrame(new_data, index=[0])

er.add_keys(df)

###############################################################################
# Using the get_keys method
# ------------
# This method returns a DataFrame of key_name, resource_table_idx, entity_id,
# and entity_uri. You can either have a single key object,
# a list of key objects, or leave the input paramters empty to return all.

# Single Key
# er.get_keys(keys=er.get_key('human'))

# # List of Specific Keys
# er.get_keys(keys=[er.get_key('human'), er.get_key('gorilla')])

# All Keys
er.get_keys()

,key_name,resources_idx,entity_id,entity_uri
0,human,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...


In [7]:
er.keys.to_dataframe()

,key
0,human
1,gorilla
2,human


In [14]:
for i in er.keys.to_dataframe():
    print(i)

key


In [10]:
er.keys